In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

c:\Users\victo\anaconda3\envs\montecarlo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def successor(X, s, r, delta, m, T):
    h = T/m
    z = np.random.normal()
    return X * np.exp(h * (r - delta - (s**2)/2) + s * np.sqrt(h) * z)

def black_scholes(X0, s, r, m, T):
    X = [X0]
    for i in range(m):
        X.append(successor(X[-1], s, r, m, T))
    return X

def put_func(a, K):
    """
    Payoff function for a Bermudan put option
    """
    return np.maximum(K - a, 0)

def call_func(a, K):
    """
    Payoff function for a Bermudan call option
    """
    return np.maximum(a - K, 0)

def high_estimator(payoff_func, current, nextt, K, r, h):
    b = len(nextt)
    discount_factor = np.exp(-r*h)
    return max(payoff_func(current, K), discount_factor * np.sum(nextt)/b)

def low_estimator(payoff_func, current, nextt, K, r, h):
    b = len(nextt)
    payoff = payoff_func(current, K)
    discount_factor = np.exp(-r*h)
    v_ = np.zeros(b)
    somme = np.sum(nextt[1:])

    if discount_factor * somme/(b-1) <= payoff:
        v_[0] = payoff
    else:
        v_[0] = discount_factor * nextt[0]

    for k in range(1, b):
        somme += nextt[k - 1] - nextt[k]
        if discount_factor * somme/(b-1) <= payoff:
            v_[k] = payoff
        else:
            v_[k] = discount_factor * nextt[k]
    return np.sum(v_)/b

In [7]:
def random_tree(X0, m, b, s, r, delta, K, T, payoff_func):
    w = np.zeros(m + 1).astype(int)
    X = np.zeros((b, m + 1))
    v = np.zeros((b, m + 1))
    V = np.zeros((b, m + 1))

    X[0, 0] = X0
    v[0, 0] = X0
    V[0, 0] = X0

    for j in range(1, m + 1):
        X[0, j] = successor(X[0, j-1], s, r, delta, m, T)
        w[j] = 0
    j = m

    while j >= 0:
        if j == m and w[j] < b - 1:
            v[w[j], j] = payoff_func(X[w[j], j], K)
            V[w[j], j] = payoff_func(X[w[j], j], K)
            X[w[j] + 1, j] = successor(X[w[j - 1], j - 1], s, r, delta, m, T)
            w[j] = w[j] + 1
        
        elif j == m and w[j] == b - 1:
            v[w[j], j] = payoff_func(X[w[j], j], K)
            V[w[j], j] = payoff_func(X[w[j], j], K)
            w[j] = 0
            j = j - 1

        elif j < m and w[j] < b - 1:
            v[w[j], j] = low_estimator(payoff_func, X[w[j], j], v[:, j+1], K, r, T/m)
            V[w[j], j] = high_estimator(payoff_func, X[w[j], j], V[:, j+1], K, r, T/m)

            if j > 0:
                X[w[j] + 1, j] = successor(X[w[j - 1], j - 1], s, r, delta, m, T)
                w[j] = w[j] + 1
                for i in range(j + 1, m + 1):
                    X[0, i] = successor(X[w[i - 1], i - 1], s, r, delta, m, T)
                    w[i] = 0
                j = m
            else:
                break

        elif j < m and w[j] == b - 1:
            v[w[j], j] = low_estimator(payoff_func, X[w[j], j], v[:, j+1], K, r, T/m)
            V[w[j], j] = high_estimator(payoff_func, X[w[j], j], V[:, j+1], K, r, T/m)
            w[j] = 0
            j = j - 1

    return max(v[0, 0], max(K - X0, 0)), V[0, 0]

In [8]:
def monte_carlo(n, X0, m, b, s, r, delta, K, T, payoff_func):
    eff_low = []
    eff_high = []
    for i in tqdm(range(n)):
        tree = random_tree(X0, m, b, s, r, delta, K, T, payoff_func)
        eff_low.append(tree[0])
        eff_high.append(tree[1])
    return np.array(eff_low), np.array(eff_high)

In [9]:
n = 100
X0 = 100
m = 3
delta = 0.05
b = 5
s = 0.2
r = 0.1
K = 100
T = 1
eff_low, eff_high = monte_carlo(n, X0, m, b, s, r, delta, K, T, put_func)

100%|██████████| 100/100 [00:00<00:00, 218.84it/s]


In [11]:
mean_v = np.mean(eff_low)
mean_V = np.mean(eff_high)

std_v = 1/(n-1) * np.sum((eff_low - mean_v)**2)
std_V = 1/(n-1) * np.sum((eff_high - mean_V)**2)

z = 1.96

std_errlow = z * std_v / np.sqrt(n)
std_errhigh = z * std_V / np.sqrt(n)
print(f'90% confidence interval: [{(mean_v - std_errlow):.4f}, {(mean_V + std_errhigh):.4f}]')

true_value = 5.57

print(f'Low estimator: {mean_v:.4f}')
print(f'High estimator: {mean_V:.4f}')
print('Std error for the low estimator:', std_errlow)
print('Std for the high estimator:', std_errhigh)
print(f'Point estimate: {0.5 * (mean_v + mean_V):.4f}')
print(f'Error: {(abs(0.5 * (mean_v + mean_V) - true_value)/true_value)*100:.2f}%')

90% confidence interval: [3.8577, 7.4251]
Low estimator: 5.0220
High estimator: 6.1488
Std error for the low estimator: 1.1642145804915875
Std for the high estimator: 1.2763003383639844
Point estimate: 5.5854
Error: 0.28%
